<a href="https://colab.research.google.com/github/englissi/englissi/blob/main/PDF%EC%9A%94%EC%95%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install gradio PyMuPDF transformers


In [11]:

import fitz  # PyMuPDF
import gradio as gr
from transformers import pipeline
import re

# 요약을 위한 모델 로드
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text("text") + "\n"
    return text

def find_section(text, section_title):
    # 정규 표현식을 사용하여 섹션 제목을 찾습니다.
    pattern = re.compile(r'(?i)^.*{}.*$'.format(section_title), re.MULTILINE)
    matches = list(pattern.finditer(text))
    if not matches:
        return None
    start_idx = matches[0].start()
    end_idx = text.find('\n\n', start_idx)
    if end_idx == -1:
        end_idx = len(text)
    section_text = text[start_idx:end_idx].strip()
    return section_text

def summarize_section(text, section_title, max_length=150):
    try:
        section_text = find_section(text, section_title)
        if section_text:
            summary = summarizer(section_text, max_length=max_length, min_length=30, do_sample=False)
            return summary[0]['summary_text']
        return f"Section '{section_title}' not found."
    except Exception as e:
        return f"Error processing section '{section_title}': {str(e)}"

def process_pdf(file):
    try:
        text = extract_text_from_pdf(file.name)
    except Exception as e:
        return [f"Error extracting text from PDF: {str(e)}"] * 3

    abstract_summary = summarize_section(text, "abstract")
    research_question_summary = summarize_section(text, "research question")
    results_summary = summarize_section(text, "results")

    return [abstract_summary, research_question_summary, results_summary]

# Gradio 인터페이스 설정
interface = gr.Interface(
    fn=process_pdf,
    inputs=gr.File(label="Upload PDF"),
    outputs=[
        gr.Textbox(label="Abstract Summary"),
        gr.Textbox(label="Research Question Summary"),
        gr.Textbox(label="Results Summary")
    ]
)

# 인터페이스 실행
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f9492b07a8337bc985.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
